# Lesson 5: Expanding the spaCy NLP Pipeline with Custom Components


Welcome to this lesson on expanding the Natural Language Processing (NLP) pipeline with custom components using the spaCy library. Today, we're going to focus on adding extensions in two ways: using a pipeline component or using a getter for precomputing results. You'll learn when to use each method and practice creating meaningful custom components.

## Understanding spaCy Extensions

Extensions in spaCy are an efficient and flexible system for adding extra functionality to the built-in Doc, Token, and Span objects, as well as some other classes such as Language and Vocab. They can be used to add more information to a Token, for example, the length of the sentence where the token is found.

Getter-based extensions are recommended when the attribute computation is straightforward, efficient, and highly dependent on the single Token instance. Such extensions are dynamically computed at the time of access, ensuring up-to-date and context-specific information without upfront computational overhead.

Let's look at an example of a getter-based extension using the Reuters text corpus.

```python
import spacy
from spacy.tokens import Token
from nltk.corpus import reuters

nlp = spacy.load("en_core_web_sm")

# Adding extensions with a getter
Token.set_extension("sentence_len", getter=lambda token: len(token.sent))

texts = reuters.raw(categories=['crude', 'coffee', 'gold'])[0:5000]
doc = nlp(texts)
```

Here, we created a simple getter-based extension that computes the length of the sentence in which each token is present.

## Accessing Extensions Created with a Getter

To access the `sentence_len` extension for each token, use the following approach:

```python
# Accessing sentence length information
for token in doc:
    print(f"{token.text}: {token._.sentence_len}")

# Example output
# JAPAN: 51
# TO: 51
# REVISE: 51
# LONG: 51
```

## Creating a Phonetic Key Extension

Now, let's add a more linguistically meaningful extension, which computes a simple linguistic feature.

Consider phonetic similarity between words. For the sake of simplicity, we'll create a phonetic key consisting of the first two consonants of the word, or if they don't exist, the first two characters. Remember, real phonetic comparison would be much more elaborate and language-dependent.

```python
def get_phonetic_key(token):
    non_vowels = [ch for ch in token.text.lower() if ch not in 'aeiou']
    return ''.join(non_vowels[:2]) if len(non_vowels) > 1 else token.text.lower()[:2]

Token.set_extension('phonetic_key', getter=get_phonetic_key)

texts = reuters.raw(categories=['crude', 'coffee', 'gold'])[0:5000]
doc = nlp(texts)
```

## Accessing the Phonetic Key Extension

After creating the phonetic key extension, you can access it for each token as follows:

```python
# Accessing the phonetic_key extension:
for token in doc:
    print(f"{token.text}: {token._.phonetic_key}")

# Example output
# JAPAN: jp
# TO: to
# REVISE: rv
# LONG: ln
```

This segment demonstrates how to use getter-based extensions effectively and retrieve the additional data they provide.

## Understanding spaCy Pipeline Components

In spaCy, a pipeline component is a function that is handed a Doc object, performs a specific operation on it, and then returns the modified Doc. Pipeline components prove useful in adding more complex extensions, especially when the computation is intricate or depends on other Tokens. This function is executed automatically as part of the spaCy pipeline when we run the nlp text processing on a document. This means that during the nlp processing of a text, each pipeline component is invoked in sequence, allowing the modifications or enhancements implemented in the component to be computed and applied to the Doc object in real-time. This design ensures that by the time the full processing is complete, all custom operations defined in the pipeline components have been performed, integrating seamlessly with spaCy's built-in processes for a holistic NLP solution.

Consider this example where we use a pipeline component to calculate the sentence length for each Token:

```python
import spacy
from spacy.language import Language
from spacy.tokens import Token
from nltk.corpus import reuters

nlp = spacy.load("en_core_web_sm")

Token.set_extension('sentence_len', default=None)

@Language.component("sentence_len_component")
def sentence_len_component(doc): 
    for sent in doc.sents:
        sent_len = len(sent)
        for token in sent:
            token._.set('sentence_len', sent_len)
    return doc

nlp.add_pipe("sentence_len_component", last=True)

texts = reuters.raw(categories=['crude', 'coffee', 'gold'])[0:5000]
doc = nlp(texts)
```

The `sentence_len_component` calculates the length of each sentence in the Doc and associates this length with every Token within that sentence. This is achieved through `token._.set('sentence_len', sent_len)`, where `token._.set(...)` is used to dynamically assign the computed sentence length to each Token.

The `Language.component` decorator is a powerful feature used to craft custom pipeline components capable of computing and assigning sentence lengths to tokens. Defining a pipeline component is only the first step; integrating it into the spaCy NLP pipeline is a seamless process facilitated by the `add_pipe` method. The first argument of `add_pipe` determines the component to incorporate into the pipeline. For our scenario, we're including the "sentence_len_component". The `last=True` parameter, passed to `add_pipe`, ensures this component is appended at the end of the pipeline. Positioning it last is crucial for ensuring that all preceding analyses and modifications have been completed on the Doc object, which allows our custom component to operate with the most current version of the document, incorporating any updates made by earlier components. This strategic placement maximizes the effectiveness and accuracy of the sentence length computation, as it relies on the context established by the pipeline's prior stages.

With this arrangement, each Token is now enriched with data regarding its sentence's length, providing a deeper level of linguistic insight for our analyses. Such enhancements to the spaCy NLP pipeline facilitate the creation of more sophisticated and tailored natural language processing applications.

## Accessing Custom Extensions from Pipeline Components

To retrieve the additional information assigned by our `sentence_len_component` to each token, you can use:

```python
# Accessing sentence length information from the pipeline component
for token in doc:
    print(f"{token.text}: {token._.sentence_len}")

# Example output
# JAPAN: 51
# TO: 51
# REVISE: 51
# LONG: 51
```

This retrieves the dynamically assigned sentence length for each token, showcasing how to interact with data generated by pipeline components.

## Lesson Summary and Practice

In this lesson, you've learned two different ways to add custom linguistic features to the spaCy NLP pipeline: using a getter or a pipeline component. By segregating the instruction on how to access custom extensions and components right after their introduction, the lesson aims to reinforce the learning by immediate application of the concepts.

In the upcoming practice exercises, you will have the chance to reinforce these concepts by applying them to real-life scenarios. Through practice, you'll be able to gain a more profound understanding of how these components can enhance your NLP tasks. Unlock the potential of custom pipeline components in spaCy and revolutionize your linguistic analysis!


## Modify Phonetic Key Function in spaCy

Great effort, Stellar Navigator! Let's tweak the get_phonetic_key function in the code. Replace the vowels used to generate the phonetic key with non-vowel characters, also known as consonants. Run your code, and observe how the phonetic keys change.


```python
import spacy
from spacy.tokens import Token
from nltk.corpus import reuters

nlp = spacy.load("en_core_web_sm")

def get_phonetic_key(token):
    vowels = [ch for ch in token.text.lower() if ch in 'aeiou']
    return ''.join(vowels[:3]) if len(vowels) > 2 else token.text.lower()[:3]

Token.set_extension('phonetic_key', getter=get_phonetic_key)

texts = reuters.raw(categories=['crude', 'coffee', 'gold'])[0:5000]
doc = nlp(texts)

for token in doc[:10]:  
    if not token.is_punct and not token.is_stop:
        print(f"{token}: {token._.phonetic_key}")


```
To modify the `get_phonetic_key` function to generate phonetic keys based on consonants instead of vowels, you can replace the logic that extracts vowels with one that extracts consonants. Here’s the updated code:

```python
import spacy
from spacy.tokens import Token
from nltk.corpus import reuters

nlp = spacy.load("en_core_web_sm")

def get_phonetic_key(token):
    # Extract consonants instead of vowels
    consonants = [ch for ch in token.text.lower() if ch not in 'aeiou']
    return ''.join(consonants[:3]) if len(consonants) > 2 else token.text.lower()[:3]

Token.set_extension('phonetic_key', getter=get_phonetic_key)

texts = reuters.raw(categories=['crude', 'coffee', 'gold'])[0:5000]
doc = nlp(texts)

for token in doc[:10]:  
    if not token.is_punct and not token.is_stop:
        print(f"{token}: {token._.phonetic_key}")
```

### Explanation of Changes:

- The `get_phonetic_key` function now extracts consonants from the token's text instead of vowels. It uses a list comprehension to filter out characters that are not vowels.
- The function returns the first three consonants if there are more than two; otherwise, it returns the first three characters of the token in lowercase.

### Example Output:

When you run this modified code, you will see the phonetic keys generated based on the consonants of the first ten tokens in the selected text. The output will look something like this (actual output may vary based on the content of the Reuters dataset):

```
the: the
crude: crd
oil: ol
prices: prc
rose: rs
...
```

This change will allow you to observe how the phonetic keys differ when generated from consonants instead of vowels.Executed 1st Code Block



## Implement Verb Count Pipeline Component

Outstanding work, Stellar Navigator! Now, let's jump to the next stage. Fill in the blanks to create a pipeline component that counts the number of verbs in each sentence and adds this count to each token in the sentence. A verb in a Token can be identified by checking if Token.pos_ represents a 'VERB'.

```python
import spacy
from spacy.tokens import Token
from spacy.language import Language
from nltk.corpus import reuters
import nltk

nltk.download('reuters')

nlp = spacy.load("en_core_web_sm")

Token.set_extension('verb_count', default=None)

@Language.component(____)
def ____(___): 
    for sent in doc.sents:
        verb_count = ____([token for token in sent if token.____ == 'VERB'])
        for token in ____:
            token._.____(____, verb_count)
    return doc

nlp.add_pipe(____, last=True) # Ensure 'count_verbs' comes after all the other standard components

texts = reuters.raw(categories=['crude', 'coffee', 'gold'])[0:5000]
doc = nlp(texts)

for token in doc[:100]:
    if not token.is_punct and not token.is_stop:
        print(f"{token}: Verb count in its sentence is {token._.verb_count}")

```

I'll help you fill in the blanks to create a pipeline component that counts verbs in sentences. Here's the completed code:

import spacy
from spacy.tokens import Token
from spacy.language import Language
from nltk.corpus import reuters
import nltk

nltk.download('reuters')

nlp = spacy.load("en_core_web_sm")

Token.set_extension('verb_count', default=None)

@Language.component("count_verbs")  # Name of the component
def count_verbs(doc):  # Function name and doc parameter
    for sent in doc.sents:
        verb_count = len([token for token in sent if token.pos_ == 'VERB'])  # Count verbs using len()
        for token in sent:  # Iterate through sentence tokens
            token._.verb_count = verb_count  # Set the custom extension value
    return doc

nlp.add_pipe("count_verbs", last=True)  # Add component name

texts = reuters.raw(categories=['crude', 'coffee', 'gold'])[0:5000]
doc = nlp(texts)

for token in doc[:100]:
    if not token.is_punct and not token.is_stop:
        print(f"{token}: Verb count in its sentence is {token._.verb_count}")

Let's break down the filled blanks:

1. `"count_verbs"` - The name of the component in the decorator
2. `count_verbs` - The function name
3. `doc` - The parameter that receives the processed document
4. `len` - Function to count the number of verbs in the list comprehension
5. `pos_` - The attribute that identifies the part of speech tag
6. `sent` - To iterate through tokens in the current sentence
7. `verb_count` - Setting the custom extension value
8. `"count_verbs"` - The component name in add_pipe()

This pipeline component will:
1. Process each sentence in the document
2. Count the number of verbs in each sentence
3. Add that count to each token's custom extension 'verb_count'
4. Return the processed document with the new information attached to each token

The final output will show each non-punctuation, non-stop word token along with the number of verbs in its containing sentence.


## Creating a Vowel Detection Custom Extension in spaCy

In this task, your objective is to develop a feature that evaluates whether each token includes any vowels and integrate this functionality as a custom extension in spaCy. Proceed to test your implementation with tokens drawn from a portion of the Reuters corpus.

```python
import spacy
from spacy.tokens import Token
from nltk.corpus import reuters

nlp = spacy.load("en_core_web_sm")

# TODO: Define the getter 'has_vowel' that checks if a token contains a vowel.

# TODO: Register 'has_vowel' as a custom attribute using Token.set_extension here.

texts = reuters.raw(categories=['crude', 'coffee', 'gold'])[0:5000]
doc = nlp(texts)

for token in doc[:50]:  
  if not token.is_punct and not token.is_stop:
    print(f"{token}: Contains a vowel? {token._.has_vowels}")


```

Good job, Stellar Navigator! For the final challenge, your goal is to write a component that counts the total number of tokens in a sentence that share the same POS (Part-Of-Speech) with a specific token. You will assign this value to each token in the sentence using the same_pos_count extension. Best of luck!

```python
import spacy
from spacy.tokens import Token
from spacy.language import Language
from nltk.corpus import reuters

nlp = spacy.load("en_core_web_sm")

# TODO: Add 'same_pos_count' extension to the Token class.

# TODO: Create a component named "count_same_pos" that counts the total number of tokens in the sentence 
# that share the same POS with the specific token. Assign it to each token in the sentence using the 'same_pos_count' extension.

# TODO: Add the "count_same_pos" component to the pipeline after all the other standard components

# Load text from the Reuters dataset
texts = reuters.raw(categories=['crude', 'coffee', 'gold'])[0:5000]
doc = nlp(texts)

# TODO: Iterate over tokens and print count of tokens of the same POS in its sentence for each token.


```

Here's how you can implement a spaCy component that counts the total number of tokens in a sentence that share the same part of speech (POS) with a specific token. The count will be assigned to each token using a custom extension called `same_pos_count`. Below is the completed code:

```python
import spacy
from spacy.tokens import Token
from spacy.language import Language
from nltk.corpus import reuters

nlp = spacy.load("en_core_web_sm")

# Add 'same_pos_count' extension to the Token class.
Token.set_extension('same_pos_count', default=None)

# Create a component named "count_same_pos" that counts the total number of tokens in the sentence 
# that share the same POS with the specific token.
@Language.component("count_same_pos")
def count_same_pos(doc):
    for sent in doc.sents:
        for token in sent:
            same_pos_count = sum(1 for t in sent if t.pos_ == token.pos_)  # Count tokens with the same POS
            token._.same_pos_count = same_pos_count  # Assign the count to the token's extension
    return doc

# Add the "count_same_pos" component to the pipeline after all the other standard components
nlp.add_pipe("count_same_pos", last=True)

# Load text from the Reuters dataset
texts = reuters.raw(categories=['crude', 'coffee', 'gold'])[0:5000]
doc = nlp(texts)

# Iterate over tokens and print count of tokens of the same POS in its sentence for each token.
for token in doc[:50]:
    if not token.is_punct and not token.is_stop:
        print(f"{token}: Count of same POS tokens in its sentence is {token._.same_pos_count}")
```

### Explanation of the Code:

1. **Custom Extension Registration**: The `Token.set_extension` method is used to create a custom attribute `same_pos_count`, which will hold the count of tokens with the same POS.

2. **Component Definition**: The `count_same_pos` function iterates through each sentence in the document. For each token in the sentence, it counts how many tokens share the same POS using a generator expression and assigns this count to the token's `same_pos_count` extension.

3. **Pipeline Integration**: The `count_same_pos` component is added to the spaCy pipeline, ensuring it runs after all other standard components.

4. **Output**: The code processes a portion of the Reuters corpus and prints the count of tokens with the same POS for each non-punctuation, non-stop word token in the first 50 tokens.

This implementation will effectively allow you to analyze the distribution of POS tags within sentences and provide insights into the structure of the text.
